<a href="https://colab.research.google.com/github/emmacionca/data_curation_and_analysis/blob/main/eac4697_data_curation_and_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
try:
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup
    print ("APIs already installed and imported")
except:
    !pip install pandas
    import pandas as pd
    !pip install beautifulsoup4
    from bs4 import BeautifulSoup
    import requests
    print ("APIs not found. Installed them and imported")



APIs already installed and imported


Check if APIs are already installed. If not, install and import them.

In [77]:
webpage = requests.get("https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population")
print ("Opened webpage")

soup = BeautifulSoup(webpage.content, "html.parser")
print ("Parsed webpage")

Opened webpage
Parsed webpage


Open webpage — store in variable.

Initialize BeautifulSoup object to read and parse webpage.

In [78]:
# Checking if the page has a table with the class 'wikitable', 'sortable' bc for some reason this one works
print('Classes of each table:')
for table in soup.find_all('table'):
    print(table.get('class'))

Classes of each table:
['wikitable', 'sortable']
['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
['nowraplinks', 'mw-collapsible', 'autocollapse', 'navbox-inner']


In [80]:
# Creating list with all tables
all_tables = soup.find_all('table')

#  Looking for the table with the classes 'wikitable' and 'sortable'
table = soup.find('table', class_='wikitable sortable')


In [81]:
messy_data = []

for row in table:
    line = row.text
    messy_data.append(line)
print(messy_data)

['\n', '\n\nCountry / Dependency\nPopulation\n% ofworld\nDate\nSource (official or fromthe United Nations)\n\n\n\n–\n\nWorld\n\n8,063,588,000\n100%\n4 Oct 2023\n\nUN projection[3]\n\n\n\n1\n\n\xa0China\n\n1,411,750,000\n17.5%\n31 Dec 2022\n\nOfficial estimate[4]\n[b]\n\n\n2\n\n\xa0India\n\n1,392,329,000\n17.3%\n1 Mar 2023\n\nOfficial projection[5]\n[c]\n\n\n3\n\n\xa0United States\n\n335,461,000\n4.2%\n4 Oct 2023\n\nNational population clock[7]\n[d]\n\n\n4\n\n\xa0Indonesia\n\n278,696,200\n3.5%\n1 Jul 2023\n\nNational annual projection[8]\n\n\n\n5\n\n\xa0Pakistan\n\n241,499,431\n3.0%\n1 Mar 2023\n\n2023 census result[9]\n[e]\n\n\n6\n\n\xa0Nigeria\n\n216,783,400\n2.7%\n21 Mar 2022\n\nOfficial projection[10]\n\n\n\n7\n\n\xa0Brazil\n\n203,062,512\n2.5%\n1 Aug 2022\n\n2022 census result[11]\n\n\n\n8\n\n\xa0Bangladesh\n\n169,828,911\n2.1%\n14 Jun 2022\n\n2022 census result[12]\n\n\n\n9\n\n\xa0Russia\n\n146,424,729\n1.8%\n1 Jan 2023\n\nOfficial estimate[13]\n[f]\n\n\n10\n\n\xa0Mexico\n\n129,20

Find all tables in the webpage and append data to an empty list to store it.

In [82]:
column_names = ['Country / Dependency', 'Population', '% ofworld', 'Date', 'Source (official or fromthe United Nations)']
#column_names = ['No.', 'Athlete', 'Nation', 'Sport', 'Years', 'Games', 'Gender', 'Gold,' 'Silver', 'Bronze', 'Total']
cleaned_data = []

for l in messy_data:
    entries = l.split("\n")
    row = {} # empty dictionary for every row
    for entry in entries:
        for column_name in column_names:
            if column_name in entry:
                entry = entry.replace(column_name,"")
                row[column_name] = entry
                break # stops the inner loop here because we already find a match
    cleaned_data.append(row)

# Sanity check - let's print the first 5 rows
print(cleaned_data[:5])

[{}, {'Country / Dependency': '', 'Population': '', '% ofworld': '', 'Date': '', 'Source (official or fromthe United Nations)': ''}]


In [83]:
cleaned_data = pd.DataFrame(cleaned_data)

# Print a few elements in the dataframe
cleaned_data

,Country / Dependency,Population,% ofworld,Date,Source (official or fromthe United Nations)
0,NaN,NaN,NaN,NaN,NaN
1,,,,,


In [85]:
cleaned_data.to_csv("Sovereign_states_and_dependencies_by_population.csv")

From here the CSV file will be used to create a data visualization.